### Sentiment analysis of movies

In [8]:
!pip install streamlit scikit-learn pandas datasets joblib matplotlib seaborn


  Using cached streamlit-1.44.1-py3-none-any.whl (9.8 MB)
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.0 MB/s eta 0:00:00a 0:00:01
  Using cached GitPython-3.1.44-py3-none-any.whl (207 kB)
  Using cached altair-5.5.0-py3-none-any.whl (731 kB)
  Using cached protobuf-5.29.4-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
  Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
  Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl (79 kB)
  Using cached click-8.1.8-py3-none-any.whl (98 kB)
  Using cached scipy-1.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  Usin

In [2]:
!pip install googletrans==4.0.0-rc1 datasets

  Preparing metadata (setup.py) ... done
  Using cached datasets-3.5.0-py3-none-any.whl (491 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 530.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 555.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 809.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.8 MB/s eta 0:00:00
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached aiohttp-3.11.16-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached huggingface_hub-0

### Importing modules


In [9]:
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import matplotlib.pyplot as plt
import seaborn as sns


##### Getting data from huggingface 

In [10]:
# Load dataset
dataset = load_dataset("sara-nabhani/ML-news-sentiment")
df = dataset['train'].to_pandas()

sentiment_train.csv:   0%|          | 0.00/627k [00:00<?, ?B/s]

sentiment_test.csv:   0%|          | 0.00/70.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4551 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/506 [00:00<?, ? examples/s]

In [12]:
df

,text,labels,label_name
0,Finnish airline Finnair is starting the tempor...,0,negative
1,The corresponding increase in the share capita...,1,neutral
2,In the third quarter of fiscal 2008 Efore swun...,0,negative
3,"ALEXANDRIA , Va. , Oct. 15 -- Aaron Moss of Ha...",1,neutral
4,Vaisala Oyj Stock exchange release 26.03.2010 ...,1,neutral
...,...,...,...
4546,The company 's main shareholders Lihakunta and...,1,neutral
4547,According to Nordic financial group Nordea 's ...,2,positive
4548,A 2001 agreement expired earlier this month .,1,neutral
4549,"The customer is then forwarded to the site , a...",1,neutral


In [14]:
# Prepare data
X = df['text']
y = df['labels']  # Assuming labels are 0:negative, 1:neutral, 2:positive


In [15]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### TFIDF vectorization pipeline

In [16]:
# Create model pipelines
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, sublinear_tf=True)

In [17]:
# Model 1: Logistic Regression
lr_pipe = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression(
        class_weight='balanced',
        solver='saga',
        multi_class='multinomial',
        max_iter=1000))
])

In [18]:

# Model 2: Support Vector Machine
svm_pipe = Pipeline([
    ('tfidf', tfidf),
    ('clf', SVC(
        class_weight='balanced',
        kernel='linear',
        probability=True))
])

In [19]:
# Train and save models
for name, model in [('Logistic Regression', lr_pipe), 
                   ('SVM', svm_pipe)]:
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    joblib.dump(model, f'{name.lower().replace(" ", "_")}_model.joblib')
    
    # Generate metrics
    y_pred = model.predict(X_test)
    print(f"\n{name} Performance:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

    # Save confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Neutral', 'Positive'],
                yticklabels=['Negative', 'Neutral', 'Positive'])
    plt.title(f'{name} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'{name.lower().replace(" ", "_")}_cm.png')
    plt.close()

Training Logistic Regression...


/home/vishrutk/movie_sentiment_analysis/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Logistic Regression Performance:
Accuracy: 0.7618
              precision    recall  f1-score   support

           0       0.60      0.72      0.66       120
           1       0.82      0.83      0.83       536
           2       0.71      0.63      0.67       255

    accuracy                           0.76       911
   macro avg       0.71      0.73      0.72       911
weighted avg       0.76      0.76      0.76       911

Training SVM...

SVM Performance:
Accuracy: 0.7508
              precision    recall  f1-score   support

           0       0.55      0.67      0.60       120
           1       0.83      0.82      0.83       536
           2       0.70      0.64      0.67       255

    accuracy                           0.75       911
   macro avg       0.69      0.71      0.70       911
weighted avg       0.76      0.75      0.75       911

